In [1]:
from __future__ import annotations

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from torch.distributions.normal import Normal

import gymnasium as gym

plt.rcParams["figure.figsize"] = (10,5)

In [2]:
## Parametrized Policy Network

class PolicyNetwork(nn.Module):

    def __init__(self, obs_space_dims, action_space_dims):
        super().__init__()

        hidden_space1 = 16
        hidden_space2 = 32

        self.shared_net = nn.Sequential(
            nn.Linear(obs_space_dims,hidden_space1),
            nn.Tanh(),
            nn.Linear(hidden_space1,hidden_space2),
            nn.Tanh()
        )

        self.policy_mean_net = nn.Sequential(
            nn.Linear(hidden_space2,action_space_dims)
        )

        self.policy_stddev_net = nn.Sequential(
            nn.Linear(hidden_space2,action_space_dims)
        )

    def forward(self, x):
        shared_features = self.shared_net(x.float())
        action_means = self.policy_mean_net(shared_features)
        action_stddevs = torch.log(
            1+torch.exp(self.policy_stddev_net(shared_features))
        )

        return action_means, action_stddevs

In [3]:
##REINFORCE
class REINFOCE:

    def __init__(self, obs_space_dims,action_space_dims):
        self.learning_rate = 1e-4
        self.gamma = 0.99
        self.eps = 1e-6

        self.net = PolicyNetwork(obs_space_dims, action_space_dims)
        self.optimizer = torch.optim.AdamW(self.net.parameters(), lr=self.learning_rate)

        self.probs = []
        self.rewards = []

    def sample_action(self, state):
        state = torch.tensor(np.array([state]))
        action_means, action_stddevs = self.net(state)

        distrib = Normal(action_means[0]+self.eps, action_stddevs[0]+self.eps)
        action = distrib.sample()
        prob = distrib.log_prob(action)
        
        action = action.numpy()

        self.probs.append(prob)

        return action
    
    def update(self):
        running_g = 0
        gs = []

        for R in self.rewards[::-1]:
            running_g = R+self.gamma*running_g
            gs.insert(0,running_g)

        deltas = torch.tensor(gs)

        loss = 0
        for log_prob, delta in zip(self.probs, deltas):
            loss += log_prob.mean()*delta*(-1)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.probs = []
        self.rewards = []



In [5]:
env = gym.make("InvertedPendulum-v4",render_mode="human")
wrapped_env = gym.wrappers.RecordEpisodeStatistics(env,50)

total_num_episodes = int(5e3)
obs_space_dims = env.observation_space.shape[0]
action_space_dims = env.action_space.shape[0]
rewards_over_seeds = []

for seed in [1,2,3,4,5,8]:
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    agent = REINFOCE(obs_space_dims,action_space_dims)
    reward_over_episodes=[]

    for episode in range(total_num_episodes):
        obs, info = wrapped_env.reset(seed=seed)
        done = False

        while not done:
            action = agent.sample_action(obs)
            obs, reward, terminated, truncated, info = wrapped_env.step(action)
            agent.rewards.append(reward)

            done = terminated or truncated

        reward_over_episodes.append(wrapped_env.return_queue[-1])
        agent.update()

        if episode % 1000 == 0:
            avg_reward = int(np.mean(wrapped_env.return_queue))
            print("Episode:", episode, "Average Reward:", avg_reward)

    rewards_over_seeds.append(reward_over_episodes)

Episode: 0 Average Reward: 8


c:\Users\Fede\anaconda3\envs\tf\lib\site-packages\glfw\__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


KeyboardInterrupt: 

In [ ]:
#Plot Learning Curve
rewards_to_plot = [[reward[0] for reward in rewards] for rewards in rewards_over_seeds]
df1 = pd.DataFrame(rewards_to_plot).melt()
df1.rename(columns={"variable":"episodes", "value":"rewards"}, inplace=True)
sns.set(style="darkgrid",context="talk", palette="rainbow")
sns.lineplot(x="episodes", y="reward", data=df1).set(
    title="REINFORCE for InvertedPendulum-4"
)
plt.show()

ValueError: Could not interpret value `reward` for parameter `y`